In [308]:
import fastai; fastai.__version__

'1.0.43.post1'

In [309]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
import string

# create data

In [310]:
path = untar_data(URLs.PETS, dest='/data/fastai/')

path.ls()

[PosixPath('/data/fastai/oxford-iiit-pet/other'),
 PosixPath('/data/fastai/oxford-iiit-pet/annotations'),
 PosixPath('/data/fastai/oxford-iiit-pet/models'),
 PosixPath('/data/fastai/oxford-iiit-pet/images')]

In [311]:
path_anno = path/'annotations'
path_img = path/'images'

In [312]:
# fastai
pat = r'/([^/]+)_\d+.jpg$'
pat = re.compile(pat)
def _get_label(fn): 
    if isinstance(fn, Path): fn = fn.as_posix()
    res = pat.search(str(fn))
    assert res,f'Failed to find "{pat}" in "{fn}"'
    return res.group(1)

In [313]:
imgs = np.random.choice([o for o in path_img.ls() if o.suffix=='.jpg'], 100)

np.random.seed(42)
cat1 = [_get_label(o) for o in imgs]
cat2 = np.random.choice(list(['A', 'B']), len(imgs))
mcat1 = [np.random.choice(list(['x', 'y', 'z']), np.random.choice(range(3)), replace=False) for _ in range(len(imgs))]
mcat2 = [np.random.choice(list(['ccc', 'sss', 'nnn', 'www']), np.random.choice(range(4)), replace=False) 
         for _ in range(len(imgs))]
num1 = [np.random.randn() for _ in range(len(imgs))]
num2 = [np.random.rand() for _ in range(len(imgs))]

df_full = pd.DataFrame({'img': [o.stem for o in imgs], 
                  'cat1': cat1, 'cat2': cat2, 
                   'mcat1': [" ".join(o) for o in mcat1],
                   'mcat2': [" ".join(o) for o in mcat2],
                   'num1': num1,
                   'num2': num2,
                  })
df_full['is_valid'] = np.random.choice([True,False], len(df_full), p=[0.2, 0.8])

In [314]:
df_full.shape

(100, 8)

In [315]:
df_full.sample(10)

,img,cat1,cat2,mcat1,mcat2,num1,num2,is_valid
27,havanese_139,havanese,B,y,www ccc nnn,-1.040850,0.511657,False
44,Abyssinian_25,Abyssinian,B,,,-0.107985,0.687166,True
3,Ragdoll_56,Ragdoll,A,z x,,0.733173,0.400300,False
60,beagle_149,beagle,A,x,www nnn,-0.656529,0.473970,False
15,chihuahua_190,chihuahua,A,z x,www,-0.103841,0.347619,False
83,scottish_terrier_64,scottish_terrier,B,x,ccc,0.116411,0.805865,False
63,British_Shorthair_40,British_Shorthair,A,x z,nnn ccc,-0.671024,0.479582,False
49,beagle_119,beagle,B,y,nnn ccc,1.232514,0.354147,False
93,american_pit_bull_terrier_35,american_pit_bull_terrier,B,x y,sss,-0.144049,0.005300,False
18,scottish_terrier_36,scottish_terrier,B,x y,ccc sss,-0.701813,0.534424,False


In [316]:
tfms = get_transforms()

# single task (using existing functionalities)

## single class

In [317]:
df=df_full[['img', 'cat1', 'is_valid']]
df.head()

,img,cat1,is_valid
0,scottish_terrier_17,scottish_terrier,False
1,keeshond_169,keeshond,False
2,great_pyrenees_190,great_pyrenees,False
3,Ragdoll_56,Ragdoll,False
4,Ragdoll_152,Ragdoll,False


### databunch factory method

In [280]:
data = ImageDataBunch.from_df(path, df, folder='images', suffix='.jpg', size=64)

/data/anaconda3/envs/fa1043/lib/python3.7/site-packages/fastai/data_block.py:486: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
yorkshire_terrier, Ragdoll
  if getattr(ds, 'warn', False): warn(ds.warn)


In [281]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [282]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate
1,4.018491,3.704471,0.833333


### data block

In [256]:
data = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='cat1')
        .transform(tfms, size=64)).databunch()

/data/anaconda3/envs/fa1043/lib/python3.7/site-packages/fastai/data_block.py:486: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
Bombay
  if getattr(ds, 'warn', False): warn(ds.warn)


In [257]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [258]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,error_rate
1,4.233921,4.636864,1.000000


## multiclass 

In [318]:
df=df_full[['img', 'mcat1', 'is_valid']]
df.head()

,img,mcat1,is_valid
0,scottish_terrier_17,x z,False
1,keeshond_169,x,False
2,great_pyrenees_190,x,False
3,Ragdoll_56,z x,False
4,Ragdoll_152,,False


### databunch factory method

In [273]:
data = ImageDataBunch.from_df(path, df, folder='images', suffix='.jpg', size=64, label_delim=' ')

In [274]:
learn = create_cnn(data, models.resnet18)
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [275]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss
1,0.868255,0.864794


### data block

In [276]:
data = (ImageItemList.from_df(df, path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='mcat1', label_delim=' ')
        .transform(tfms, size=64)).databunch()

In [277]:
learn = create_cnn(data, models.resnet18)
learn.loss_func

FlattenedLoss of BCEWithLogitsLoss()

In [278]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss
1,0.840250,0.740074


## regression

### databunch factory method

In [244]:
# doesn't work
# data = ImageDataBunch.from_df(path, df[['img', 'num1']], folder='images', suffix='.jpg', size=64)

### data block

In [245]:
data = (ImageItemList.from_df(df[['img', 'num1', 'is_valid']], path, folder='images', suffix='.jpg')
        .split_from_df(col='is_valid')
        .label_from_df(cols='num1')
        .transform(tfms, size=64)).databunch()

In [247]:
learn = create_cnn(data, models.resnet18, metrics=mse)
learn.loss_func

FlattenedLoss of MSELoss()

In [248]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,mean_squared_error
1,2.089933,1.220198,1.220198


# multi task

https://docs.fast.ai/tutorial.itemlist.html

## 2 single classes

In [319]:
df=df_full[['img', 'cat1', 'cat2', 'is_valid']]
df.head()

,img,cat1,cat2,is_valid
0,scottish_terrier_17,scottish_terrier,A,False
1,keeshond_169,keeshond,B,False
2,great_pyrenees_190,great_pyrenees,A,False
3,Ragdoll_56,Ragdoll,A,False
4,Ragdoll_152,Ragdoll,A,False


```
class Category(ItemBase):
    "Basic class for single classification labels."
    def __init__(self,data,obj): self.data,self.obj = data,obj
    def __int__(self): return int(self.data)
    def __str__(self): return str(self.obj)

class MultiCategory(ItemBase):
    "Basic class for multi-classification labels."
    def __init__(self,data,obj,raw): self.data,self.obj,self.raw = data,obj,raw
    def __str__(self): return ';'.join([str(o) for o in self.obj])

class FloatItem(ItemBase):
    "Basic class for float items."
    def __init__(self,obj): self.data,self.obj = np.array(obj).astype(np.float32),obj
    def __str__(self): return str(self.obj)
```

https://docs.fast.ai/tutorial.itemlist.html#Example:-ImageTuple
```
class ImageTuple(ItemBase):
    def __init__(self, img1, img2):
        self.img1,self.img2 = img1,img2
        self.obj,self.data = (img1,img2),[-1+2*img1.data,-1+2*img2.data]
```

In [161]:
class MultitaskTarget(ItemBase):
    "Basic class for multitask target."
    def __init__(self,target): 
        self.target = target
        self.data = []
        
    def __str__(self): 
        return str(self.target)

def multitask_loss():
    return 0

class MultitaskTargetList():
    def __init__(self, items:Iterator, 
                 classes1:Collection=None, 
                 classes1:Collection=None, 
                 mclasses1:Collection=None, 
                 mclasses1:Collection=None, 
                 label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)
        self.loss_func = multitask_loss

    def get(self, i):
        o = self.items[i]
        if o is None: return None
        return MultitaskTarget(o, self.classes[o])
    
    

In [162]:
target = df.to_dict('records')[np.random.choice(len(df))]
del target['img']
target

{'cat1': 'chihuahua',
 'cat2': 'O',
 'mcat1': 's e z',
 'mcat2': 'abc xyz',
 'num1': -0.10838220224884176,
 'num2': 0.38568197098798007}

In [163]:
MultitaskTarget(target)

MultitaskTarget {'cat1': 'chihuahua', 'cat2': 'O', 'mcat1': 's e z', 'mcat2': 'abc xyz', 'num1': -0.10838220224884176, 'num2': 0.38568197098798007}

```
class CategoryList(CategoryListBase):
    "Basic `ItemList` for single classification labels."
    _processor=CategoryProcessor
    def __init__(self, items:Iterator, classes:Collection=None, label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)
        self.loss_func = CrossEntropyFlat()

    def get(self, i):
        o = self.items[i]
        if o is None: return None
        return Category(o, self.classes[o])

    def analyze_pred(self, pred, thresh:float=0.5): return pred.argmax()

    def reconstruct(self, t):
        return Category(t, self.classes[t])
```